# XGBoost 모델학습을 위한 전처리
- 백테스트를 위해 시가 종가 데이트 다 살려야함
- 정규화할때 필요한 컬럼들만 리스트 제외시키는 방법으로 학습시켜야함.
- data split


# 코인별 기간이 각기 달라도 병합시 xgboost가 잘학습되는 이유
- pd.concat(all_data, axis=0, ignore_index=True): 각 코인의 모든 행을 단순히 세로로 쌓기만 함
- Date 컬럼이 있어서 시간 정보는 보존됨
- XGBoost는 tabular 데이터이므로 각 행이 독립적인 관측치로 취급

# 주의
- 결과: BTC 데이터가 압도적으로 많아서 BTC 패턴에 과적합 위험
- 시대적 편향 : 2017-2020: 주로 BTC, ETH 같은 메이저 코인
- 2021-2024: 다양한 알트코인 추가
- 학습 데이터: 과거 = 메이저코인, 최근 = 알트코인
- 예측할 때: 최근 패턴을 학습했는데, 메이저코인에는 적용이 안 될 수 있음

# data load

In [1]:
import pandas as pd
import os
import numpy as np

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import joblib

import warnings
import utils as U
import strategy as stg
warnings.filterwarnings('ignore')

In [2]:
# 라벨링하기 전 path
# data_path="/workspace/AI모델/projects/coin/data/labeling/00"

# 라벨링  + 아직 추가 피쳐엔지니어링은 안한 데이터
data_path = "/workspace/AI모델/projects/coin/data/labeling/01"

# 메이저종목만
# data_path = "/workspace/AI모델/projects/coin/data/labeling/m_01"

# 저장된 데이터 불러오기 load_data
crypto_data, excluded_info, summary_info = U.load_cleaned_crypto_data(data_path)

# 불러온 데이터 확인
print(f"로딩된 종목 수: {len(crypto_data)}")
for symbol, df in list(crypto_data.items())[:3]:
    print(f"{symbol}: {len(df)}행, {df.index[0].date()} ~ {df.index[-1].date()}")

# 상세 정보 출력
U.get_data_info(crypto_data, detailed=True)

# ML 데이터셋 로드
# ml_dataset = U.CryptoDataLoader.load_ml_dataset(data_path)


📂 데이터 로딩 시작: /workspace/AI모델/projects/coin/data/labeling/01
📊 발견된 파일 수: 100
✅ 성공적으로 로드된 심볼: 100개
❌ 제외된 심볼: 0개
🔥 로딩 에러: 0개
로딩된 종목 수: 100
1INCH: 1725행, 2020-12-25 ~ 2025-09-14
AAVE: 1809행, 2020-10-02 ~ 2025-09-14
ADA: 2867행, 2017-11-09 ~ 2025-09-14

📊 데이터 정보 요약
총 심볼 수: 100
컬럼 수: 203
전략 시그널 수: 41
데이터 기간: 2015-01-01 ~ 2025-09-14
평균 데이터 포인트: 2258일

📋 심볼별 상세 정보:
  1INCH: 1725행, 2020-12-25 ~ 2025-09-14
  AAVE: 1809행, 2020-10-02 ~ 2025-09-14
  ADA: 2867행, 2017-11-09 ~ 2025-09-14
  ALGO: 2278행, 2019-06-21 ~ 2025-09-14
  ANKR: 2385행, 2019-03-06 ~ 2025-09-14
  API3: 1748행, 2020-12-02 ~ 2025-09-14
  APT21794: 1062행, 2022-10-19 ~ 2025-09-14
  ARB11841: 907행, 2023-03-23 ~ 2025-09-14
  ASTR: 1336행, 2022-01-18 ~ 2025-09-14
  ATOM: 2377행, 2019-03-14 ~ 2025-09-14
  ... 및 90개 더

🎯 전략 시그널 목록:
  이동평균: 4개
  MACD: 6개
  RSI: 8개
  오실레이터: 6개
  가격패턴: 3개
  거래량: 4개
  모멘텀: 2개
  복합: 2개


# 필요한 컬럼만 정규화 및 전처리 - 모델학습용

In [3]:
stocks = crypto_data.copy()
display(stocks["BTC"].head(4))
print(stocks["BTC"].keys())

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,...,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,320.434998,320.434998,314.002991,314.248993,8036550,NaN,NaN,NaN,NaN,NaN,...,-0.098330,-0.332237,-0.307988,-0.213134,-0.162855,0.382133,0,Strong_Sell,1,Hold
2015-01-02,314.079010,315.838989,313.565002,315.032013,7860650,314.640503,NaN,NaN,NaN,NaN,...,-0.078164,-0.339442,-0.279527,-0.196891,-0.179064,0.375854,0,Strong_Sell,1,Hold
2015-01-03,314.846008,315.149994,281.082001,281.082001,33054400,298.057007,303.454336,NaN,NaN,NaN,...,-0.022364,-0.291097,-0.152457,-0.095204,-0.091326,0.529842,0,Strong_Sell,1,Hold
2015-01-04,281.145996,287.230011,257.612000,264.195007,55629100,272.638504,286.769674,293.639503,NaN,NaN,...,0.005545,-0.203849,-0.139772,-0.039736,-0.029747,0.639285,0,Strong_Sell,1,Hold


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'MA_2', 'MA_3', 'MA_4',
       'MA_5', 'MA_6',
       ...
       'Future_7d', 'Future_14d', 'Future_30d', 'Future_90d', 'Future_180d',
       'Future_365d', 'Label', 'Label_Name', 'Optimized_Label',
       'Optimized_Label_Name'],
      dtype='object', length=203)


In [4]:

# 학습에서 제외할 컬럼들 - 학습,평가시엔 Future_Label사용
### 학습코드에서 추가로 제외하므로 먼저 이정도만 제외
EXCLUDE_FROM_TRAINING = [
    'Date', 'Symbol', 'Future_Label', 'Label', 'Optimized_Label','Volatility_Signal', 
    'Volume_Breakout_Signal', 'Composite_Score',
    'Label_Name', 'Optimized_Label_Name',
    'Open', 'High', 'Low', 'Close', 'Volume',  # 원본 가격/거래량
    'Adj Close', 'Dividend', 'Stock Split',  # 기타 원본 데이터
]

# 백테스팅용 보존 컬럼들 - 백테스팅에서는 실제라벨인 Optimized_Label사용
BACKTEST_PRESERVE_COLUMNS = [
    'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 
    'Label', 'Label_Name', 'Optimized_Label', 'Optimized_Label_Name', 'Symbol'
]

# 정규화에서 제외할 Signal/Binary 변수들
SIGNAL_FEATURES = [
    # 퀀트 전략 시그널들 (0, 1, 2 값)
    'MA_Cross_3_25_Signal', 'MA_Trend_Signal', 'EMA_Cross_5_20_Signal', 'EMA_Cross_6_24_Signal',
    'Jungjin_Signal', 'MACD_Zero_Cross_Signal', 'MACD_10_20_Signal', 'MACD_15_26_Signal',
    'MACD_5_27_Signal', 'MACD_Signal_Cross', 'Bad_Market3_Signal',
    'RSI_Reversal_Signal', 'RSI_Extreme_Signal', 'RSI_Reverse_Signal', 'RSI_20_75_Signal',
    'RSI_22_78_Signal', 'RSI_30_65_Signal', 'RSI_12_50_Signal',
    'Williams_CCI_Signal', 'CCI_Oversold_Signal', 'CCI_3_Signal', 'Stoch_RSI_Combo_Signal',
    'Stoch_10_72_Signal', 'Stoch_71_31_Signal',
    'Candlestick_Signal', 'Formula3_Signal', 'Shadow_Analysis_Signal', 'Pivot_Strategy_Signal',
    'MFI_Strategy_Signal', 'MFI_25_50_Signal', 'MFI_Extreme_Signal', 'Volume_Breakout_Signal',
    'Momentum_Signal', 'Price_ROC_3_Signal', 'Volatility_Signal',
    'Final_Composite_Signal',
    
    # 이미 정규화된 형태의 변수들 (0~1 사이 또는 비율)
    'BB_Position_20',  # 0~1 사이 볼린저밴드 위치
    'Volume_Ratio',  # 현재 거래량 / 평균 거래량 비율
    'HighLow_Position_52d', 'HighLow_Position_200d',  # 0~1 사이 고저점 위치
    'Price_vs_MA7', 'Price_vs_MA20', 'Price_vs_MA50', 'Price_vs_MA100', 'Price_vs_MA200',  # 이미 비율
    
    # 수익률 변수들 (이미 비율/퍼센트 형태)
    'Return_1d', 'Return_3d', 'Return_7d', 'Return_14d', 'Return_30d', 
    'Return_90d', 'Return_180d', 'Return_365d', 'Cumulative_Return',
    
    # 카운트 변수들 (정수값이지만 범위가 한정적)
    'Buy_Signal_Count', 'Sell_Signal_Count', 'Net_Signal_Score',
    
    # 메타 변수들 (정규화 불필요)
    'Symbol', 'Date', 'Target_Label', 'Future_Return_7d',
    
    # 추후 생성될 가능성이 있는 이진/범주형 변수들
    'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end',
    'market_regime_bull', 'market_regime_bear', 'market_regime_sideways',
    'volatility_regime_high', 'volatility_regime_low'
]

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, RobustScaler
import joblib
import warnings
warnings.filterwarnings('ignore')


def debug_infinite_values(df, step_name):
    """무한값 디버깅 함수"""
    inf_cols = []
    for col in df.select_dtypes(include=[np.number]).columns:
        if np.isinf(df[col]).any():
            inf_count = np.isinf(df[col]).sum()
            inf_cols.append(f"{col}: {inf_count}개")
    
    if inf_cols:
        print(f"🚨 {step_name}에서 무한값 발견:")
        for col_info in inf_cols:
            print(f"  - {col_info}")
    return len(inf_cols) > 0

class XGBoostStockPreprocessor:
    """XGBoost 학습을 위한 종목별 정규화 전처리기"""
    
    def __init__(self, forecast_horizon=1):
        self.forecast_horizon = forecast_horizon
        self.symbol_stats = {}
        self.global_scalers = {}
        
    def calculate_symbol_statistics(self, processed_stocks):
        """종목별 기본 통계 계산"""
        print("종목별 통계 계산 중...")
        
        for symbol, df in processed_stocks.items():
            # 안전한 통계 계산 (0 방지)
            close_values = df['Close'].replace(0, np.nan).dropna()
            volume_values = df['Volume'].replace(0, np.nan).dropna() if 'Volume' in df.columns else pd.Series([1])
            
            stats = {
                'avg_close': close_values.mean() if len(close_values) > 0 else 1.0,
                'close_std': close_values.std() if len(close_values) > 0 else 1.0,
                'avg_volume': volume_values.mean() if len(volume_values) > 0 else 1.0,
                'volatility': close_values.pct_change().std() * np.sqrt(252) if len(close_values) > 1 else 0.1,
                'price_level': 'high' if close_values.mean() > 100 else 'low'
            }
            
            # NaN 방지
            for key, value in stats.items():
                if pd.isna(value) or value == 0:
                    if 'avg' in key or 'std' in key:
                        stats[key] = 1.0
                    else:
                        stats[key] = 0.1
                        
            self.symbol_stats[symbol] = stats
        
        print(f"  처리된 종목 수: {len(self.symbol_stats)}")
    
    def create_price_features(self, df, symbol):
        """가격 기반 피쳐 생성 (로그 수익률 중심) - 무한값 방지"""
        df_processed = df.copy()
        
        # 0값들을 매우 작은 값으로 대체 (무한값 방지)
        price_columns = ['Close', 'Open', 'High', 'Low']
        for col in price_columns:
            if col in df.columns:
                df_processed[col] = df_processed[col].replace(0, 1e-8)
        
        # 1. 로그 수익률 계산 (안전하게)
        for col in price_columns:
            if col in df.columns:
                # 로그 수익률 (안전한 계산)
                price_ratio = df_processed[col] / df_processed[col].shift(1)
                price_ratio = price_ratio.replace([0, np.inf, -np.inf], np.nan)
                price_ratio = price_ratio.clip(lower=1e-10, upper=1e10)  # 극값 제한
                df_processed[f'{col}_log_return'] = np.log(price_ratio)
                
                # 단순 수익률
                df_processed[f'{col}_return'] = df_processed[col].pct_change()
        
        # 2. 이동평균들을 현재가 대비 비율로 변환 (새로운 MA 기간들 포함)
        ma_columns = [col for col in df.columns if col.startswith('MA_') and col != 'MACD']
        for col in ma_columns:
            if col in df.columns:
                # 0값 방지
                ma_values = df_processed[col].replace(0, 1e-8)
                close_values = df_processed['Close'].replace(0, 1e-8)
                
                df_processed[f'{col}_ratio'] = ma_values / close_values
                df_processed[f'{col}_distance'] = (close_values - ma_values) / close_values
        
        # 3. EMA들도 비율로 변환 (새로운 EMA 기간들 포함)
        ema_columns = [col for col in df.columns if col.startswith('EMA_')]
        for col in ema_columns:
            if col in df.columns:
                ema_values = df_processed[col].replace(0, 1e-8)
                close_values = df_processed['Close'].replace(0, 1e-8)
                df_processed[f'{col}_ratio'] = ema_values / close_values
        
        # 4. 볼린저 밴드를 상대적 위치로 변환
        if 'BB_Upper_20' in df.columns and 'BB_Lower_20' in df.columns:
            # 볼린저밴드 폭 계산 (0 방지)
            bb_width = df_processed['BB_Upper_20'] - df_processed['BB_Lower_20']
            bb_width = bb_width.replace(0, 1e-8)
            
            # 볼린저밴드 내 상대적 위치
            df_processed['BB_relative_position'] = (
                (df_processed['Close'] - df_processed['BB_Lower_20']) / bb_width
            ).fillna(0.5)
            
            # 각 밴드 대비 거리 (안전하게)
            close_values = df_processed['Close'].replace(0, 1e-8)
            df_processed['BB_upper_distance'] = (df_processed['BB_Upper_20'] - close_values) / close_values
            df_processed['BB_lower_distance'] = (close_values - df_processed['BB_Lower_20']) / close_values
        
        # 5. 거래량 정규화 (안전하게)
        if 'Volume' in df.columns:
            avg_volume = self.symbol_stats[symbol]['avg_volume']
            volume_values = df_processed['Volume'].replace(0, 1e-8)
            
            df_processed['Volume_normalized'] = volume_values / avg_volume
            df_processed['Volume_log_ratio'] = np.log(volume_values / avg_volume + 1e-8)
        
        # 무한값 및 극값 처리
        numeric_columns = df_processed.select_dtypes(include=[np.number]).columns
        for col in numeric_columns:
            df_processed[col] = df_processed[col].replace([np.inf, -np.inf], np.nan)
            if df_processed[col].notna().sum() > 0:
                # 극값 클리핑 (99.9% / 0.1% percentile)
                p99 = df_processed[col].quantile(0.999)
                p01 = df_processed[col].quantile(0.001)
                if not pd.isna(p99) and not pd.isna(p01):
                    df_processed[col] = df_processed[col].clip(lower=p01, upper=p99)
        
        return df_processed
    
    def create_technical_features(self, df):
        """기술적 지표 기반 추가 피쳐 생성"""
        df_enhanced = df.copy()
        
        # 1. RSI 기반 피쳐 (다양한 기간의 RSI 처리)
        rsi_columns = [col for col in df.columns if col.startswith('RSI_')]
        for rsi_col in rsi_columns:
            if rsi_col in df.columns:
                df_enhanced[f'{rsi_col}_overbought'] = (df[rsi_col] > 70).astype(int)
                df_enhanced[f'{rsi_col}_oversold'] = (df[rsi_col] < 30).astype(int)
                df_enhanced[f'{rsi_col}_normalized'] = (df[rsi_col] - 50) / 50  # -1 to 1 범위
        
        # 2. MACD 기반 피쳐 (다양한 MACD 변형들 처리)
        macd_columns = [col for col in df.columns if col.startswith('MACD') and not col.endswith('_Signal')]
        for macd_col in macd_columns:
            signal_col = f'{macd_col}_Signal'
            if macd_col in df.columns and signal_col in df.columns:
                df_enhanced[f'{macd_col}_above_signal'] = (df[macd_col] > df[signal_col]).astype(int)
                df_enhanced[f'{macd_col}_signal_distance'] = df[macd_col] - df[signal_col]
        
        # 3. 스토캐스틱 기반 피쳐 (다양한 기간 처리)
        stoch_k_columns = [col for col in df.columns if col.startswith('Stoch_K_')]
        for k_col in stoch_k_columns:
            period = k_col.replace('Stoch_K_', '')
            d_col = f'Stoch_D_{period}'
            if k_col in df.columns and d_col in df.columns:
                df_enhanced[f'Stoch_{period}_K_above_D'] = (df[k_col] > df[d_col]).astype(int)
                df_enhanced[f'Stoch_{period}_overbought'] = ((df[k_col] > 80) & (df[d_col] > 80)).astype(int)
                df_enhanced[f'Stoch_{period}_oversold'] = ((df[k_col] < 20) & (df[d_col] < 20)).astype(int)
        
        # 4. Williams %R 기반 피쳐 (다양한 기간 처리)
        williams_columns = [col for col in df.columns if col.startswith('Williams_R_')]
        for wr_col in williams_columns:
            if wr_col in df.columns:
                df_enhanced[f'{wr_col}_overbought'] = (df[wr_col] > -20).astype(int)
                df_enhanced[f'{wr_col}_oversold'] = (df[wr_col] < -80).astype(int)
        
        # 5. CCI 기반 피쳐 (다양한 기간 처리)
        cci_columns = [col for col in df.columns if col.startswith('CCI_')]
        for cci_col in cci_columns:
            if cci_col in df.columns:
                df_enhanced[f'{cci_col}_overbought'] = (df[cci_col] > 100).astype(int)
                df_enhanced[f'{cci_col}_oversold'] = (df[cci_col] < -100).astype(int)
        
        # 6. 변동성 피쳐 (안전하게)
        if 'Close_log_return' in df_enhanced.columns:
            # 실현 변동성 (rolling) - NaN 처리
            log_returns = df_enhanced['Close_log_return'].fillna(0)
            df_enhanced['realized_volatility_5'] = log_returns.rolling(5, min_periods=1).std()
            df_enhanced['realized_volatility_20'] = log_returns.rolling(20, min_periods=1).std()
            
            # 변동성 레짐 (고변동성/저변동성)
            vol_20 = df_enhanced['realized_volatility_20'].fillna(0)
            vol_median = vol_20.median() if vol_20.sum() > 0 else 0
            df_enhanced['high_volatility_regime'] = (vol_20 > vol_median).astype(int)
        
        # 7. 추세 강도 피쳐
        if 'MA_20_ratio' in df_enhanced.columns and 'MA_50_ratio' in df_enhanced.columns:
            df_enhanced['trend_strength'] = df_enhanced['MA_20_ratio'] - df_enhanced['MA_50_ratio']
            df_enhanced['strong_uptrend'] = (df_enhanced['trend_strength'] > 0.02).astype(int)
            df_enhanced['strong_downtrend'] = (df_enhanced['trend_strength'] < -0.02).astype(int)
        
        return df_enhanced
    
    def create_time_features(self, df):
        """시간 기반 피쳐 생성"""
        df_time = df.copy()
        
        # Date 컬럼 확실히 생성
        if isinstance(df.index, pd.DatetimeIndex):
            df_time['Date'] = df.index
            dates = df.index
        elif 'Date' in df.columns:
            dates = pd.to_datetime(df['Date'])
        else:
            return df_time
        
        df_time['year'] = dates.year
        df_time['month'] = dates.month
        df_time['quarter'] = dates.quarter
        df_time['day_of_week'] = dates.dayofweek
        df_time['is_month_start'] = dates.is_month_start.astype(int)
        df_time['is_month_end'] = dates.is_month_end.astype(int)
        df_time['is_quarter_start'] = dates.is_quarter_start.astype(int)
        df_time['is_quarter_end'] = dates.is_quarter_end.astype(int)
        
        # 순환 인코딩 (월, 요일)
        df_time['month_sin'] = np.sin(2 * np.pi * df_time['month'] / 12)
        df_time['month_cos'] = np.cos(2 * np.pi * df_time['month'] / 12)
        df_time['dow_sin'] = np.sin(2 * np.pi * df_time['day_of_week'] / 7)
        df_time['dow_cos'] = np.cos(2 * np.pi * df_time['day_of_week'] / 7)
        
        return df_time
    
    def prepare_single_symbol(self, df, symbol):
        """단일 종목 데이터 전처리"""
        # 1. 가격 기반 피쳐 생성
        df_price = self.create_price_features(df, symbol)
        
        # 2. 기술적 지표 피쳐 생성
        df_technical = self.create_technical_features(df_price)
        
        # 3. 시간 피쳐 생성
        df_complete = self.create_time_features(df_technical)
        
        # 4. 타겟 라벨 생성 (Future_Label 생성)
        if 'Optimized_Label' in df_complete.columns:
            df_complete['Future_Label'] = df_complete['Optimized_Label'].shift(-self.forecast_horizon)
        else:
            print("no Optimized_label... : 2차라벨 생성이 안되었습니다.")
        
        # 5. 종목 정보 추가
        df_complete['Symbol'] = symbol
        
        return df_complete
    
    def apply_global_scaling(self, combined_df):
        """전체 데이터에 대한 글로벌 스케일링"""
        df_scaled = combined_df.copy()
        
        # 스케일링 전 무한값 및 이상치 처리
        print("무한값 및 이상치 처리 중...")
        
        # 1. 무한값을 NaN으로 변환
        df_scaled = df_scaled.replace([np.inf, -np.inf], np.nan)
        
        # 2. 매우 큰 값들 클리핑 (99.9% percentile 기준)
        numeric_cols = df_scaled.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if df_scaled[col].notna().sum() > 0:  # 데이터가 있는 경우만
                p99 = df_scaled[col].quantile(0.999)
                p01 = df_scaled[col].quantile(0.001)
                if not pd.isna(p99) and not pd.isna(p01) and p99 != p01:
                    df_scaled[col] = df_scaled[col].clip(lower=p01, upper=p99)
        
        # 3. 남은 NaN 처리 (스케일링 전에)
        df_scaled = df_scaled.fillna(0)
        
        print(f"  처리 후 무한값 수: {np.isinf(df_scaled.select_dtypes(include=[np.number])).sum().sum()}")
        
        # 문자열 컬럼들 먼저 식별
        string_columns = df_scaled.select_dtypes(include=['object']).columns.tolist()
        
        # 스케일링할 피쳐 그룹 분류
        log_return_features = [col for col in df_scaled.columns if '_log_return' in col]
        return_features = [col for col in df_scaled.columns if '_return' in col and '_log_return' not in col]
        ratio_features = [col for col in df_scaled.columns if '_ratio' in col]
        distance_features = [col for col in df_scaled.columns if '_distance' in col]
        volatility_features = [col for col in df_scaled.columns if 'volatility' in col or 'Volatility' in col]
        
        # SIGNAL_FEATURES에 정의된 정규화 제외 특성들
        normalized_features = SIGNAL_FEATURES.copy()
        
        # 나머지 기술적 지표들
        exclude_cols = (EXCLUDE_FROM_TRAINING + log_return_features + return_features + 
                    ratio_features + distance_features + volatility_features + normalized_features +
                    ['year', 'month', 'quarter', 'day_of_week'] + string_columns)
        
        other_features = [col for col in df_scaled.columns if col not in exclude_cols]
        
        # 스케일러 초기화
        if not self.global_scalers:
            self.global_scalers = {
                'log_return_scaler': StandardScaler(),
                'return_scaler': RobustScaler(quantile_range=(1, 99)),
                'ratio_scaler': StandardScaler(),
                'distance_scaler': StandardScaler(),
                'volatility_scaler': RobustScaler(quantile_range=(5, 95)),
                'other_scaler': StandardScaler()
            }
        
        # 각 그룹별 스케일링 (안전하게)
        def safe_scaling(scaler, data, feature_names, scaler_name):
            if feature_names and len(feature_names) > 0:
                clean_data = data[feature_names].fillna(0)
                # 추가 무한값 체크
                if np.isinf(clean_data).any().any():
                    print(f"  ⚠️ {scaler_name}에서 무한값 발견, 0으로 대체")
                    clean_data = clean_data.replace([np.inf, -np.inf], 0)
                
                try:
                    scaled_values = scaler.fit_transform(clean_data)
                    data[feature_names] = scaled_values
                    print(f"  ✅ {scaler_name}: {len(feature_names)}개 피쳐 스케일링 완료")
                except Exception as e:
                    print(f"  ❌ {scaler_name} 스케일링 실패: {e}")
                    # 실패시 0으로 대체
                    data[feature_names] = 0
            return data
        
        df_scaled = safe_scaling(self.global_scalers['log_return_scaler'], df_scaled, 
                               log_return_features, "로그수익률")
        df_scaled = safe_scaling(self.global_scalers['return_scaler'], df_scaled, 
                               return_features, "일반수익률")
        df_scaled = safe_scaling(self.global_scalers['ratio_scaler'], df_scaled, 
                               ratio_features, "비율")
        df_scaled = safe_scaling(self.global_scalers['distance_scaler'], df_scaled, 
                               distance_features, "거리")
        df_scaled = safe_scaling(self.global_scalers['volatility_scaler'], df_scaled, 
                               volatility_features, "변동성")
        df_scaled = safe_scaling(self.global_scalers['other_scaler'], df_scaled, 
                               other_features, "기타지표")
        
        return df_scaled
    
    def get_training_features(self, df):
        """학습에 사용할 피쳐만 선택"""
        all_columns = df.columns.tolist()
        training_features = [col for col in all_columns if col not in EXCLUDE_FROM_TRAINING]
        return training_features
    
    def process_all_stocks(self, processed_stocks, output_dir):
        """전체 종목 처리 및 데이터 분할"""
        print("XGBoost용 데이터 전처리 시작")
        print("=" * 60)
        
        # 1. 종목별 통계 계산
        self.calculate_symbol_statistics(processed_stocks)
        
        # 2. 종목별 전처리
        all_data = []
        backtest_data = {}
        
        print("\n종목별 전처리 중...")
        for symbol, df in processed_stocks.items():
            print(f"  처리 중: {symbol}")
            
            # 백테스팅용 원본 데이터 보존 (Date 처리)
            df_reset = df.reset_index()
            if 'Date' not in df_reset.columns and 'index' in df_reset.columns:
                df_reset.rename(columns={'index': 'Date'}, inplace=True)
            
            # BACKTEST_PRESERVE_COLUMNS에 있는 컬럼들만 선택
            available_cols = [col for col in BACKTEST_PRESERVE_COLUMNS if col in df_reset.columns]
            backtest_data[symbol] = df_reset[available_cols].copy()
            
            # 학습용 데이터 전처리
            processed_df = self.prepare_single_symbol(df, symbol)
            all_data.append(processed_df)
        
        # 3. 모든 데이터 합치기
        combined_df = pd.concat(all_data, axis=0, ignore_index=True)
        print(f"\n합친 데이터 크기: {combined_df.shape}")
        
        # 4. 글로벌 스케일링
        print("글로벌 스케일링 적용 중...")
        scaled_df = self.apply_global_scaling(combined_df)
        
        # 5. 최종 결측치 및 무한값 처리
        scaled_df = scaled_df.replace([np.inf, -np.inf], np.nan)
        scaled_df = scaled_df.dropna(subset=['Future_Label'])
        
        # 최종 결측치 처리
        numeric_columns = scaled_df.select_dtypes(include=[np.number]).columns
        scaled_df[numeric_columns] = scaled_df[numeric_columns].fillna(0)
        
        print(f"최종 데이터 크기: {scaled_df.shape}")
        
        # 6. 라벨 분포 확인
        print(f"\n라벨 분포:")
        label_counts = scaled_df['Future_Label'].value_counts().sort_index()
        for label, count in label_counts.items():
            label_name = {0: 'Sell', 1: 'Hold', 2: 'Buy'}[int(label)]
            print(f"  {int(label)} ({label_name}): {count:,}개 ({count/len(scaled_df)*100:.1f}%)")
        
        # 7. 학습용 피쳐 리스트
        training_features = self.get_training_features(scaled_df)
        print(f"\n학습용 피쳐 수: {len(training_features)}")
        
        # 8. 데이터 저장
        os.makedirs(output_dir, exist_ok=True)
        
        # 전체 데이터 (학습용)
        scaled_df.to_csv(os.path.join(output_dir, 'xgboost_training_data.csv'), index=False)
        
        # 백테스팅용 데이터 (종목별 개별 파일)
        backtest_dir = os.path.join(output_dir, 'backtest_data')
        os.makedirs(backtest_dir, exist_ok=True)
        
        for symbol, data in backtest_data.items():
            data.to_csv(os.path.join(backtest_dir, f'{symbol}_backtest.csv'), index=False)
        
        # 전처리기 저장
        joblib.dump(self, os.path.join(output_dir, 'xgboost_preprocessor.pkl'))
        
        # 피쳐 리스트 저장
        with open(os.path.join(output_dir, 'training_features.txt'), 'w') as f:
            for feature in training_features:
                f.write(f"{feature}\n")
        
        print(f"\n데이터 저장 완료:")
        print(f"  - 학습용: {output_dir}/xgboost_training_data.csv")
        print(f"  - 백테스팅용: {backtest_dir}/")
        print(f"  - 전처리기: {output_dir}/xgboost_preprocessor.pkl")
        print(f"  - 피쳐 리스트: {output_dir}/training_features.txt")
        
        return scaled_df, backtest_data, training_features

def split_time_series_data(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """시계열 데이터 시간순 분할"""
    
    # Date 기준 정렬
    if 'Date' in df.columns:
        df_sorted = df.sort_values('Date').copy()
    else:
        df_sorted = df.copy()
    
    n = len(df_sorted)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    
    train_df = df_sorted.iloc[:train_end].copy()
    val_df = df_sorted.iloc[train_end:val_end].copy()
    test_df = df_sorted.iloc[val_end:].copy()
    
    print(f"\n데이터 분할 결과:")
    print(f"  훈련: {len(train_df):,}개 ({len(train_df)/n*100:.1f}%)")
    print(f"  검증: {len(val_df):,}개 ({len(val_df)/n*100:.1f}%)")
    print(f"  테스트: {len(test_df):,}개 ({len(test_df)/n*100:.1f}%)")
    
    if 'Date' in df.columns:
        print(f"  기간: {train_df['Date'].min().date()} ~ {test_df['Date'].max().date()}")
    
    return train_df, val_df, test_df

def filter_target_symbols(processed_stocks, target_symbols=None):
    """
    타겟 종목들만 필터링
    
    Parameters:
    -----------
    processed_stocks : dict
        전체 종목별 데이터프레임
    target_symbols : list or None
        선택할 종목 리스트. None이면 전체 사용
        예: ['BTC', 'ETH', 'SOL'] (-USD 제외)
    
    Returns:
    --------
    dict : 필터링된 종목별 데이터프레임
    """
    if target_symbols is None:
        print("🔄 전체 종목 사용")
        return processed_stocks
    
    # -USD 제거하여 정규화
    normalized_targets = []
    for symbol in target_symbols:
        clean_symbol = symbol.replace('-USD', '').replace('_USDT', '').replace('USDT', '')
        normalized_targets.append(clean_symbol)
    
    # 사용 가능한 종목들 확인
    available_symbols = list(processed_stocks.keys())
    filtered_stocks = {}
    found_symbols = []
    missing_symbols = []
    
    for target in normalized_targets:
        # 정확히 일치하는 심볼 찾기
        matched = False
        for available in available_symbols:
            # 다양한 패턴 매칭
            clean_available = available.replace('-USD', '').replace('_USDT', '').replace('USDT', '')
            if clean_available == target or available == target:
                filtered_stocks[available] = processed_stocks[available]
                found_symbols.append(f"{target} → {available}")
                matched = True
                break
        
        if not matched:
            missing_symbols.append(target)
    
    # 결과 출력
    print(f"🎯 타겟 종목 필터링 결과:")
    print(f"  요청된 종목: {len(target_symbols)}개")
    print(f"  발견된 종목: {len(found_symbols)}개")
    
    if found_symbols:
        print("  ✅ 매칭된 종목들:")
        for match in found_symbols:
            print(f"    - {match}")
    
    if missing_symbols:
        print(f"  ❌ 찾을 수 없는 종목들: {missing_symbols}")
        print(f"  💡 사용 가능한 종목들: {list(processed_stocks.keys())[:10]}...")
    
    return filtered_stocks

# 메인 실행 함수
def prepare_xgboost_data(processed_stocks, output_dir='./data/xgboost', forecast_horizon=1, target_symbols=None):
    """
    XGBoost 학습을 위한 완전한 데이터 전처리
    
    Parameters:
    -----------
    processed_stocks : dict
        종목별 데이터프레임 (Final_Composite_Signal 포함)
    output_dir : str
        출력 디렉토리
    forecast_horizon : int
        예측 기간 (일)
    target_symbols : list or None
        선택할 종목 리스트. None이면 전체 사용
        예: ['BTC', 'ETH', 'SOL', 'DOGE'] (-USD는 자동 제거됨)
    
    Returns:
    --------
    dict : (학습용_데이터, 백테스팅용_데이터, 피쳐_리스트)
    """
    
    # 1. 타겟 종목 필터링
    filtered_stocks = filter_target_symbols(processed_stocks, target_symbols)
    
    if not filtered_stocks:
        raise ValueError("❌ 선택된 종목이 없습니다!")
    
    # 2. 전처리기 초기화
    preprocessor = XGBoostStockPreprocessor(forecast_horizon=forecast_horizon)
    
    # 3. 전체 데이터 처리
    scaled_df, backtest_data, training_features = preprocessor.process_all_stocks(
        filtered_stocks, output_dir
    )
    
    # 4. 시계열 분할
    train_df, val_df, test_df = split_time_series_data(scaled_df)
    
    # 5. 분할된 데이터 저장
    train_df.to_csv(os.path.join(output_dir, 'train_data.csv'), index=False)
    val_df.to_csv(os.path.join(output_dir, 'val_data.csv'), index=False)
    test_df.to_csv(os.path.join(output_dir, 'test_data.csv'), index=False)
    
    return {
        'train': train_df,
        'val': val_df, 
        'test': test_df,
        'backtest_data': backtest_data,
        'training_features': training_features,
        'preprocessor': preprocessor
    }

# 실행
- 여기서 training_features.txt에 컬럼리스트를 적어두는데 모델학습 시, 이 컬럼들을 기준으로 불러오게 한 후 추가로 제외해야한다.
- B00_01_train코드에서 셀3번 참고

In [6]:
processed_stocks = crypto_data.copy()
# 전체
output_dir='/workspace/AI모델/projects/coin/data/xgboost_B'
# 메이저만
# output_dir='/workspace/AI모델/projects/coin/data/xgboost_B_major'

result = prepare_xgboost_data(
    processed_stocks=processed_stocks,
    output_dir=output_dir,  # 출력 경로 지정
    forecast_horizon=1,  # 5라면 5일 후 예측, 0이면 당장 오늘의 시그널.
    target_symbols=None  # 전체종목
)

# 2. 결과 접근
train_df = result['train']
val_df = result['val'] 
test_df = result['test']
backtest_data = result['backtest_data']  # 종목별 원본 데이터
training_features = result['training_features']

🔄 전체 종목 사용
XGBoost용 데이터 전처리 시작
종목별 통계 계산 중...
  처리된 종목 수: 100

종목별 전처리 중...
  처리 중: 1INCH
  처리 중: AAVE
  처리 중: ADA
  처리 중: ALGO
  처리 중: ANKR
  처리 중: API3
  처리 중: APT21794
  처리 중: ARB11841
  처리 중: ASTR
  처리 중: ATOM
  처리 중: AVAX
  처리 중: AXS
  처리 중: BAL
  처리 중: BAND
  처리 중: BAT
  처리 중: BCH
  처리 중: BNB
  처리 중: BNT
  처리 중: BSV
  처리 중: BTC
  처리 중: CAKE
  처리 중: CFG
  처리 중: CHZ
  처리 중: CKB
  처리 중: COTI
  처리 중: CRO
  처리 중: CRV
  처리 중: CVC
  처리 중: CVX
  처리 중: DAI
  처리 중: DASH
  처리 중: DOGE
  처리 중: DOT
  처리 중: DYDX
  처리 중: EGLD
  처리 중: ENJ
  처리 중: ETC
  처리 중: ETH
  처리 중: FET
  처리 중: FIL
  처리 중: FLOW
  처리 중: FTM
  처리 중: FXS
  처리 중: GNO
  처리 중: GRT6719
  처리 중: HBAR
  처리 중: ICP
  처리 중: IMX10603
  처리 중: INJ
  처리 중: IOTA
  처리 중: KCS
  처리 중: KEEP
  처리 중: KNC
  처리 중: LEO
  처리 중: LPT
  처리 중: LRC
  처리 중: LTC
  처리 중: MANA
  처리 중: MNT27075
  처리 중: NEAR
  처리 중: NEO
  처리 중: NMR
  처리 중: OKB
  처리 중: OP
  처리 중: QNT
  처리 중: QTUM
  처리 중: REQ
  처리 중: RLC
  처리 중: RNDR
  처리 중: RSR
  처리 중: RUNE
  처리 중: SAND
  처리 중: SHI

In [9]:

import pandas as pd

# 모든 열과 행을 표시하도록 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [8]:
display(train_df.head(8),train_df.tail(8))
display(val_df.head(8),val_df.tail(8))
display(test_df.head(8),test_df.tail(4))

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

In [10]:
print(train_df.columns.tolist())
print(val_df.columns.tolist())
print(test_df.columns.tolist())
print(len(train_df.keys()))
print(len(val_df.keys()))
print(len(test_df.keys()))

# 중복 컬럼 확인
print(f"Train 중복 컬럼: {len(train_df.columns) - len(set(train_df.columns))}")
print(f"Val 중복 컬럼: {len(val_df.columns) - len(set(val_df.columns))}")
print(f"Test 중복 컬럼: {len(test_df.columns) - len(set(test_df.columns))}")

['Open', 'High', 'Low', 'Close', 'Volume', 'MA_2', 'MA_3', 'MA_4', 'MA_5', 'MA_6', 'MA_7', 'MA_8', 'MA_10', 'MA_13', 'MA_16', 'MA_20', 'MA_25', 'MA_29', 'MA_50', 'MA_60', 'MA_100', 'MA_120', 'MA_200', 'EMA_3', 'EMA_4', 'EMA_5', 'EMA_6', 'EMA_7', 'EMA_10', 'EMA_12', 'EMA_20', 'EMA_24', 'EMA_26', 'EMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram', 'MACD_10_20', 'MACD_10_20_Signal', 'MACD_15_26', 'MACD_15_26_Signal', 'MACD_5_27', 'MACD_5_27_Signal', 'RSI_4', 'RSI_5', 'RSI_6', 'RSI_7', 'RSI_12', 'RSI_14', 'RSI_20', 'RSI_21', 'BB_Middle_20', 'BB_Upper_20', 'BB_Lower_20', 'BB_Width_20', 'BB_Position_20', 'Return_1d', 'Return_3d', 'Return_7d', 'Return_14d', 'Return_30d', 'Return_90d', 'Return_180d', 'Return_365d', 'Cumulative_Return', 'Volatility_7d', 'Volatility_14d', 'Volatility_30d', 'Volatility_90d', 'Volume_MA_7', 'Volume_MA_20', 'Volume_MA_50', 'Volume_Ratio', 'Volume_Price_Trend', 'True_Range', 'ATR_14', 'ATR_Percent_14', 'Stoch_K_1', 'Stoch_D_1', 'Stoch_K_3', 'Stoch_D_3', 'Stoch_K_4', 

# 메이저만 저장

In [11]:
### -USD는 자동으로 제거됨
major_coins = [
    'BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'SOL-USD', 
    'USDC-USD', 'XRP-USD', 'DOGE-USD', 'TON11419-USD', 'ADA-USD',
    'SHIB-USD', 'AVAX-USD', 'TRX-USD', 'DOT-USD', 'BCH-USD',
    'NEAR-USD', 'MATIC-USD', 'ICP-USD', 'UNI7083-USD', 'LTC-USD'
]

In [16]:
processed_stocks = crypto_data.copy()

result = prepare_xgboost_data(
    processed_stocks=processed_stocks,
    output_dir='/workspace/AI모델/projects/coin/data/xgboost_B_major/',  # 출력 경로 지정
    target_symbols=major_coins,
    forecast_horizon=1  # 5라면 5일 후 예측, 0이면 당장 오늘의 시그널.
)

# 2. 결과 접근
train_df = result['train']
val_df = result['val'] 
test_df = result['test']
backtest_data = result['backtest_data']  # 종목별 원본 데이터
training_features = result['training_features']

🎯 타겟 종목 필터링 결과:
  요청된 종목: 20개
  발견된 종목: 19개
  ✅ 매칭된 종목들:
    - BTC → BTC
    - ETH → ETH
    -  → USDT
    - BNB → BNB
    - SOL → SOL
    - USDC → USDC
    - XRP → XRP
    - DOGE → DOGE
    - TON11419 → TON11419
    - ADA → ADA
    - SHIB → SHIB
    - AVAX → AVAX
    - TRX → TRX
    - DOT → DOT
    - BCH → BCH
    - NEAR → NEAR
    - ICP → ICP
    - UNI7083 → UNI7083
    - LTC → LTC
  ❌ 찾을 수 없는 종목들: ['MATIC']
  💡 사용 가능한 종목들: ['ADA', 'AVAX', 'BCH', 'BNB', 'BTC', 'DOGE', 'DOT', 'ETH', 'ICP', 'LTC']...
XGBoost용 데이터 전처리 시작
종목별 통계 계산 중...
  처리된 종목 수: 19

종목별 전처리 중...
  처리 중: BTC
  처리 중: ETH
  처리 중: USDT
  처리 중: BNB
  처리 중: SOL
  처리 중: USDC
  처리 중: XRP
  처리 중: DOGE
  처리 중: TON11419
  처리 중: ADA
  처리 중: SHIB
  처리 중: AVAX
  처리 중: TRX
  처리 중: DOT
  처리 중: BCH
  처리 중: NEAR
  처리 중: ICP
  처리 중: UNI7083
  처리 중: LTC

합친 데이터 크기: (47466, 386)
글로벌 스케일링 적용 중...
무한값 및 이상치 처리 중...
  처리 후 무한값 수: 0
  ✅ 로그수익률: 4개 피쳐 스케일링 완료
  ✅ 일반수익률: 4개 피쳐 스케일링 완료
  ✅ 비율: 34개 피쳐 스케일링 완료
  ✅ 거리: 26개 피쳐 스케일링 완료
  ✅ 변동성: 10개 피쳐

In [17]:
display(train_df.head(8),train_df.tail(8))
display(val_df.head(8),val_df.tail(8))
display(test_df.head(8))

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

,Open,High,Low,Close,Volume,MA_2,MA_3,MA_4,MA_5,MA_6,MA_7,MA_8,MA_10,MA_13,MA_16,MA_20,MA_25,MA_29,MA_50,MA_60,MA_100,MA_120,MA_200,EMA_3,EMA_4,EMA_5,EMA_6,EMA_7,EMA_10,EMA_12,EMA_20,EMA_24,EMA_26,EMA_50,MACD,MACD_Signal,MACD_Histogram,MACD_10_20,MACD_10_20_Signal,MACD_15_26,MACD_15_26_Signal,MACD_5_27,MACD_5_27_Signal,RSI_4,RSI_5,RSI_6,RSI_7,RSI_12,RSI_14,RSI_20,RSI_21,BB_Middle_20,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,Return_1d,Return_3d,Return_7d,Return_14d,Return_30d,Return_90d,Return_180d,Return_365d,Cumulative_Return,Volatility_7d,Volatility_14d,Volatility_30d,Volatility_90d,Volume_MA_7,Volume_MA_20,Volume_MA_50,Volume_Ratio,Volume_Price_Trend,True_Range,ATR_14,ATR_Percent_14,Stoch_K_1,Stoch_D_1,Stoch_K_3,Stoch_D_3,Stoch_K_4,Stoch_D_4,Stoch_K_5,Stoch_D_5,Stoch_K_6,Stoch_D_6,Stoch_K_14,Stoch_D_14,Williams_R_1,Williams_R_8,Williams_R_10,Williams_R_14,Williams_R,CCI_3,CCI_4,CCI_10,CCI_17,CCI_20,CCI,MFI_11,MFI_14,MFI,OSCP_11_36,OSCP_18_19,OSCP_19_20,TR,DM_Plus,DM_Minus,DI_Plus_21,DI_Minus_21,ADX_21,Price_ROC_3,STD_1,STD_20,Highest_3,Lowest_3,Highest_7,Lowest_7,Highest_8,Lowest_8,Highest_10,Lowest_10,Highest_11,Lowest_11,Highest_16,Lowest_16,Highest_17,Lowest_17,Highest_28,Lowest_28,Price_vs_MA7,Price_vs_MA20,Price_vs_MA50,Price_vs_MA100,Price_vs_MA200,HighLow_Position_52d,HighLow_Position_200d,MA_Cross_3_25_Signal,MA_Trend_Signal,EMA_Cross_5_20_Signal,EMA_Cross_6_24_Signal,Jungjin_Signal,MACD_Zero_Cross_Signal,MACD_Signal_Cross,Bad_Market3_Signal,RSI_Reversal_Signal,RSI_Extreme_Signal,RSI_Reverse_Signal,RSI_20_75_Signal,RSI_22_78_Signal,RSI_30_65_Signal,RSI_12_50_Signal,Williams_CCI_Signal,CCI_Oversold_Signal,CCI_3_Signal,Stoch_RSI_Combo_Signal,Stoch_10_72_Signal,Stoch_71_31_Signal,Candlestick_Signal,Formula3_Signal,Shadow_Analysis_Signal,Pivot,Support1,Resistance1,Pivot_Strategy_Signal,MFI_Strategy_Signal,MFI_25_50_Signal,MFI_Extreme_Signal,Volume_Breakout_Signal,Momentum_Signal,Price_ROC_3_Signal,Volatility_Signal,Composite_Signal_Avg,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Final_Composite_Signal,Price_Momentum_5,Price_Momentum_10,Price_Momentum_20,Volatility_20,Volatility_Ratio,RSI_Divergence,MACD_Cross,Support_Level,Resistance_Level,Price_Position,Technical_Score,Enhanced_Technical_Score,Composite_Score,Future_1d,Future_3d,Future_7d,Future_14d,Future_30d,Future_90d,Future_180d,Future_365d,Label,Label_Name,Optimized_Label,Optimized_Label_Name,Close_log_return,Close_return,Open_log_return,Open_return,High_log_return,High_return,Low_log_return,Low_return,MA_2_ratio,MA_2_distance,MA_3_ratio,MA_3_distance,MA_4_ratio,MA_4_distance,MA_5_ratio,MA_5_distance,MA_6_ratio,MA_6_distance,MA_7_ratio,MA_7_distance,MA_8_ratio,MA_8_distance,MA_10_ratio,MA_10_distance,MA_13_ratio,MA_13_distance,MA_16_ratio,MA_16_distance,MA_20_ratio,MA_20_distance,MA_25_ratio,MA_25_distance,MA_29_ratio,MA_29_distance,MA_50_ratio,MA_50_distance,MA_60_ratio,MA_60_distance,MA_100_ratio,MA_100_distance,MA_120_ratio,MA_120_distance,MA_200_ratio,MA_200_distance,MA_Cross_3_25_Signal_ratio,MA_Cross_3_25_Signal_distance,MA_Trend_Signal_ratio,MA_Trend_Signal_distance,EMA_3_ratio,EMA_4_ratio,EMA_5_ratio,EMA_6_ratio,EMA_7_ratio,EMA_10_ratio,EMA_12_ratio,EMA_20_ratio,EMA_24_ratio,EMA_26_ratio,EMA_50_ratio,EMA_Cross_5_20_Signal_ratio,EMA_Cross_6_24_Signal_ratio,BB_relative_position,BB_upper_distance,BB_lower_distance,Volume_normalized,Volume_log_ratio,RSI_4_overbought,RSI_4_oversold,RSI_4_normalized,RSI_5_overbought,RSI_5_oversold,RSI_5_normalized,RSI_6_overbought,RSI_6_oversold,RSI_6_normalized,RSI_7_overbought,RSI_7_oversold,RSI_7_normalized,RSI_12_overbought,RSI_12_oversold,RSI_12_normalized,RSI_14_overbought,RSI_14_oversold,RSI_14_normalized,RSI_20_overbought,RSI_20_oversold,RSI_20_normalized,RSI_21_overbought,RSI_21_oversold,RSI_21_normalized,RSI_Reversal_Signal_overbought,RSI_Reversal_Signal_oversold,RSI_Reversal_Signal_normalized,RSI_Extreme_Signal_overbought,RSI_Extreme_Signal_oversold,RSI_Extreme_Signal_normalized,RSI_Reverse_Signal_ove

In [18]:
print(train_df.columns.tolist())
print(val_df.columns.tolist())
print(test_df.columns.tolist())
print(len(train_df.keys()))
print(len(val_df.keys()))
print(len(test_df.keys()))

['Open', 'High', 'Low', 'Close', 'Volume', 'MA_2', 'MA_3', 'MA_4', 'MA_5', 'MA_6', 'MA_7', 'MA_8', 'MA_10', 'MA_13', 'MA_16', 'MA_20', 'MA_25', 'MA_29', 'MA_50', 'MA_60', 'MA_100', 'MA_120', 'MA_200', 'EMA_3', 'EMA_4', 'EMA_5', 'EMA_6', 'EMA_7', 'EMA_10', 'EMA_12', 'EMA_20', 'EMA_24', 'EMA_26', 'EMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram', 'MACD_10_20', 'MACD_10_20_Signal', 'MACD_15_26', 'MACD_15_26_Signal', 'MACD_5_27', 'MACD_5_27_Signal', 'RSI_4', 'RSI_5', 'RSI_6', 'RSI_7', 'RSI_12', 'RSI_14', 'RSI_20', 'RSI_21', 'BB_Middle_20', 'BB_Upper_20', 'BB_Lower_20', 'BB_Width_20', 'BB_Position_20', 'Return_1d', 'Return_3d', 'Return_7d', 'Return_14d', 'Return_30d', 'Return_90d', 'Return_180d', 'Return_365d', 'Cumulative_Return', 'Volatility_7d', 'Volatility_14d', 'Volatility_30d', 'Volatility_90d', 'Volume_MA_7', 'Volume_MA_20', 'Volume_MA_50', 'Volume_Ratio', 'Volume_Price_Trend', 'True_Range', 'ATR_14', 'ATR_Percent_14', 'Stoch_K_1', 'Stoch_D_1', 'Stoch_K_3', 'Stoch_D_3', 'Stoch_K_4', 